# Votes (Divisions) in the Oireachtas: 01 - Data

An API is now available, see https://data.oireachtas.ie/ for documentation.

In [1]:
# Commonly useful notebook functions:
import sys
sys.path.insert(0,'../lib')
from common import *

In [2]:
# Save json from https://api.oireachtas.ie/
# Max results available for paging through with limit & skip is 10,000 so splitting by years works best
def save_page(year, overwrite=False):
    save_page_year('divisions', year, overwrite)
    
# Entire dataset: 11475 as of 15-12-2017, starts at 1922
years = range(1922, datetime.now().year + 1)
process_with_progress(save_page, years)
print()

In [3]:
# To update after loading everything already: Grab the latest year and overwrite it:
save_page(datetime.now().year, True)

In [7]:
# Explore an example year of results (2002 has very few votes for some reason):
with lzma.open('data/year/2002.json.xz', 'rb') as f:
    preview = json.loads(f.read().decode('utf-8'))

In [8]:
RenderJSON(preview) # Preview all

In [9]:
RenderJSON(preview['results'][0]['division']) # Preview single vote

In [4]:
# Load Data from a single file and return dataframe
def divisions_df(fname):
    with lzma.open(fname, 'rb') as f:
        data = json.loads(f.read().decode('utf-8'))
        #print(fname, len(data['results']))
        
        # We don't care about the "head" part for now, just "results"
        records = json.dumps([r['division'] for r in data['results']])
        df = pd.read_json(records, orient='records', dtype=False)
        return df

In [5]:
# Process all the files and stick them together in a single dataframe:
files = glob('data/year/*.json.xz')

data_frames = process_with_progress(divisions_df, files)
df_raw = pd.concat(data_frames, axis=0)

In [6]:
# Not all the columns are useful, we can transform and drop the redundant ones:
df_raw.sample(3)

,category,chamber,date,datetime,debate,house,isBill,outcome,subject,tallies,tellers,uri,voteId
207,Division,"{'showAs': 'Seanad Éireann', 'uri': '/ie/oirea...",1991-03-13,NaT,"{'debateSection': 'dbsect_3', 'showAs': 'Order...","{'showAs': '19th Seanad', 'houseNo': '19', 'ch...",False,Carried,"{'showAs': 'Question put.', 'uri': None}","{'staonVotes': None, 'taVotes': {'showAs': 'Tá...","Tellers: Tá, Senators Wright and Fitzgerald; N...",/ie/oireachtas/division/house/seanad/19/1991-0...,vote_1
40,Division,"{'showAs': 'Dáil Éireann', 'uri': '/ie/oireach...",1982-03-24,NaT,{'showAs': 'Private Members' Business. - Emplo...,"{'chamberType': 'house', 'showAs': '23rd Dáil'...",False,Carried,"{'showAs': 'Amendment put.', 'uri': None}",{'taVotes': {'members': [{'member': {'memberCo...,"Tellers: Tá, Deputies B. Ahern and Briscoe; Ní...",/ie/oireachtas/division/house/dail/23/1982-03-...,vote_1
5,Division,"{'showAs': 'Dáil Éireann', 'uri': '/ie/oireach...",1928-11-29,NaT,"{'debateSection': 'dbsect_16', 'showAs': 'PUBL...","{'committeeCode': '', 'houseNo': '6', 'showAs'...",False,_,"{'showAs': 'Amendment put.', 'uri': None}","{'nilVotes': {'showAs': 'Níl', 'members': [{'m...",Tellers:—Tá: Deputies Boland and Allen. Níl: ...,/ie/oireachtas/division/house/dail/6/1928-11-2...,vote_2


In [7]:
flatten_columns = ['chamber','debate','house','subject'] # deal with tallies later
df_raw = flatten_dataframe_columns(df_raw, flatten_columns)

# category is always "Division", isBill is always False, not used in "Divisions"
# debate.formats.pdf, subject.uri are always empty for this data
df_raw.drop(['category','isBill','debate.formats.pdf','subject.uri'], axis=1, inplace=True)

In [8]:
df_raw.sample(3)

,chamber.showAs,chamber.uri,debate.debateSection,debate.formats.xml.uri,debate.showAs,debate.uri,house.chamberType,house.committeeCode,house.houseCode,house.houseNo,house.showAs,house.uri,subject.showAs,date,datetime,outcome,tallies,tellers,uri,voteId
150,Seanad Éireann,/ie/oireachtas/house/seanad,dbsect_6,/akn/ie/debateRecord/seanad/2010-03-31/debate/...,Lost at Sea Scheme: Motion.,/akn/ie/debateRecord/seanad/2010-03-31/debate/...,house,,seanad,23,23rd Seanad,/ie/oireachtas/house/seanad/23,"The Seanad divided: Tá,",2010-03-31,NaT,Carried,"{'taVotes': {'showAs': 'Tá', 'tally': 29, 'mem...","Tellers: Tá, Senators Niall Ó Brolcháin and Di...",/ie/oireachtas/division/house/seanad/23/2010-0...,vote_1
38,Dáil Éireann,/ie/oireachtas/house/dail,dbsect_10,/akn/ie/debateRecord/dail/1924-11-20/debate/mu...,"LOCAL GOVERNMENT BILL, 1924,—THIRD STAGE (RESU...",/akn/ie/debateRecord/dail/1924-11-20/debate/main,house,,dail,4,4th Dáil,/ie/oireachtas/house/dail/4,"The Committee divided: Tá,",1924-11-20,NaT,_,"{'nilVotes': {'tally': 37, 'members': [{'membe...",,/ie/oireachtas/division/house/dail/4/1924-11-2...,vote_1
127,Dáil Éireann,/ie/oireachtas/house/dail,dbsect_19,/akn/ie/debateRecord/dail/2005-03-01/debate/mu...,Social Welfare and Pensions Bill 2005: Report ...,/akn/ie/debateRecord/dail/2005-03-01/debate/main,house,,dail,29,29th Dáil,/ie/oireachtas/house/dail/29,"The Dáil divided: Tá,",2005-03-01,NaT,Carried,"{'staonVotes': None, 'nilVotes': {'members': [...","Tellers: Tá, Deputies Kitt and Kelleher; Níl, ...",/ie/oireachtas/division/house/dail/29/2005-03-...,vote_1


In [9]:
describe_with_top_n(df_raw, exclude_columns=['tallies'], n=50)

----------

# chamber.showAs

###  Stats:

,chamber.showAs
count,11487
unique,69
top,Dáil Éireann
freq,7348
missing,0


###  Top N:

...

# voteId

###  Stats:

,voteId
count,11487
unique,195
top,vote_1
freq,5220
missing,0


###  Top N:

,voteId
vote_1,5220
vote_2,2631
vote_3,1372
vote_4,749
vote_5,434
vote_6,268
vote_7,155
vote_8,104
vote_9,72
vote_10,50


In [10]:
df_raw.to_pickle('divisions.p.xz', compression='xz') # gzip is faster, but xz makes much smaller files for git